In [8]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
from sklearn.model_selection import train_test_split
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
import time
import datetime
from dateutil.relativedelta import relativedelta
import os #ディレクトリ作成用
import xgboost as xgb
import sys
#自作のモジュールのインポート
sys.path.append("..")
import module.master as master
import module.modeling_scores as modeling

### モデルのスコア保存のための箱作り

In [9]:
version='V4_1'#バージョン

place_master=master.get_place_master()
for place in place_master.items():
    #print(place[0],place[1],'\n')
    place_name=place[1]
    dir_path = "../../bot_database/{place_name}/model_score_{place_name}/".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
    if os.path.exists(dir_path)==False:
        os.makedirs(dir_path)
    else:
        pass


#V4_系列の特殊処理
for place in place_master.items():
    #print(place[0],place[1],'\n')
    place_name=place[1]
    dir_path = "../../bot_database/{place_name}/model_score_{place_name}/v4_score/{version}/".format(place_name=place_name,version=version)#作成したデータの書き込み先#使用するデータの読み込み
    if os.path.exists(dir_path)==False:
        os.makedirs(dir_path)
    else:
        pass

# 機能のテスト

### 今回使用していく関数を作る

## 実行する関数内で使用されている関数

In [10]:
def get_event_info(result_base_df):
    df=result_base_df.copy()
    df['date']=pd.to_datetime(df['date'])#日付が文字列なのでdateを日付型に変換
    df['year']=df['date'].dt.year
    df['month']=df['date'].dt.month
    df['day']=df['date'].dt.day

    num_date=1
    num_date_arr=[]
    last_race_date=df['date'].values[0]#前レースの日付(処理開始時用にtarainのデータの一番初めのdateを仮に入力しておく)
    for index,row in df.iterrows():
        today_date=row['date']
        if today_date==last_race_date:#同じ日のレースだったらおなじレース日を配列に追加、次の日の日付を出力（ほぼ無操作みたいなもん）
            next_date=row['date'] + datetime.timedelta(days=1)#次の日
            num_date_arr.append(num_date)
        else:#日にちが変わった時
            if today_date==next_date:#想定していた日付（次の日のレース）だったら,レース日を一日足して、そのレース日番号を加算
                num_date+=1
                num_date_arr.append(num_date)
                last_race_date=row['date']#前回レース日を上書き
                #next_date=train_df[train_df['date']==row['date'] + datetime.timedelta(days=1)]#次の日
                next_date=row['date'] + datetime.timedelta(days=1)#次の日
                #print(next_date)
            else:#想定していた日付でない(違う大会になった)場合はレース日をリセット
                num_date=1
                num_date_arr.append(num_date)
                last_race_date=row['date']#前回レース日を上書き
                #next_date=train_df[train_df['date']==row['date'] + datetime.timedelta(days=1)]#次の日
                next_date=row['date'] + datetime.timedelta(days=1)#次の日
    df['num_date']=num_date_arr

    range_races=0#大会中の取得できたレースの数
    range_date=1#大会の開催日数
    range_date_arr=[]
    range_date_arr_2=[]#for文中で繰り返し上書きさせる用の配列
    last_race_date=df['date'].values[0]#前レースの日付(処理開始時用にtrainのデータの一番初めのdateを仮に入力しておく)
    for index,row in df.iterrows():
        today_date=row['date']
        if today_date==last_race_date:#同じ日のレースだったらおなじレース日を配列に追加、次の日の日付を出力（ほぼ無操作みたいなもん）
            range_races+=1
            next_date=row['date'] + datetime.timedelta(days=1)#次の日
            #num_date_arr.append(num_date)
        else:#日にちが変わった時
            if today_date==next_date:#想定していた日付（次の日のレース）だったら,レース日を一日足して終了
                range_date+=1
                range_races+=1
                last_race_date=row['date']#前回レース日を上書き
                #next_date=train_df[train_df['date']==row['date'] + datetime.timedelta(days=1)]#次の日
                next_date=row['date'] + datetime.timedelta(days=1)#次の日次の日
            else:#想定していた日付でない(違う大会になった)場合は現在のrange_dateをもとに前の大会のレースに大会開催日数を持たせる。

                range_date_arr_2=[range_date]*range_races
                for num in range_date_arr_2:
                    range_date_arr.append(num)
                range_races=1#大会中の取得できたレースの数
                range_date=1#大会の開催日数
                last_race_date=row['date']#前回レース日を上書き
                #next_date=train_df[train_df['date']==row['date'] + datetime.timedelta(days=1)]#次の日
                next_date=row['date'] + datetime.timedelta(days=1)#次の日
    range_date_arr_2=[range_date]*range_races#最後の日は日付の変わり絵が発生しないので特別処理
    for num in range_date_arr_2:
        range_date_arr.append(num)
    df['range_date']=range_date_arr

    #四半期カラムの作成
    df['season']=df['month']
    df['season']=df['season'].replace([3,4,5],'sp')#春
    df['season']=df['season'].replace([6,7,8],'su')#夏
    df['season']=df['season'].replace([9,10,11],'au')#秋
    df['season']=df['season'].replace([12,1,2],'wi')#冬
    #df=df.drop('date',axis=1)
    return df

def pred_th_trans(pred_df,th):
    #引数として予測結果のdeと、変換したい閾値を渡す。
    trans_df=pred_df.copy()
    trans_df.loc[trans_df['pred_proba'] >= th, 'pred'] = 1
    trans_df.loc[~(trans_df['pred_proba']  >=  th), 'pred'] = 0
    return trans_df

def calc_monthly_analysis(pred_df):#予測に加えて，配当，開催情報が結合されたdfを渡すことで月ごと関連の分析を行ってくれる関数
    cols=['month','use','get','income','income_per','num_hit','buy_hit_per','mean_income','median_income']
    monthly_analysis_df= pd.DataFrame(columns=cols)#月別収益結果の入る箱
    months=pred_df['month'].value_counts(sort=False).index
    for month in months:
        monthly_df=pred_df[pred_df['month']==month].copy()
        use_m=100*monthly_df['pred'].sum()
        get_m=monthly_df['gain'].sum()
        income=get_m-use_m
        income_per=(get_m/use_m)*100
        
        
        num_hit=monthly_df['hit_flag'].sum()
        num_pred=pred_df['pred'].sum()
        buy_hit_per=(num_hit/num_pred)*100
        if num_hit==0:#警告文削除用
            mean_income=0
            median_income=0
        else:
            mean_income=monthly_df[monthly_df['hit_flag']==1]["gain"].mean()#１回の的中あたりの平均配当
            median_income=monthly_df[monthly_df['hit_flag']==1]["gain"].median()#１回の的中あたりの中央配当
        
        append_arr=[month,use_m,get_m,income,income_per,num_hit,buy_hit_per,mean_income,median_income]
        append_s=pd.Series(append_arr,index=cols)
        monthly_analysis_df=monthly_analysis_df.append(append_s, ignore_index=True)
    return monthly_analysis_df
    

### 実行する関数

In [13]:
def data_making_clustar_section_has_final(df,now_ym,range_test_m,range_final_m):#モデル関連に使用するdfの作成関数(クラスタリングあり、モータ番号、艇番号なし)（加工関数）
    #V4系列で使用する加工関数，年と月の情報を使って直近のデータを使って性能検証を行う（区間を使ってデータを作っている）
    #validデータを作成するバージョンなので実装する際はこれをそのまま使わず，final_test部分の処理は消してくだちい
    #yearが使わないと思うけど一応残してあるから邪魔だと思ったら消して下さい
    result_df=df
    result_df=result_df.drop(["racer_1_ID","racer_2_ID","racer_3_ID","racer_4_ID","racer_5_ID","racer_6_ID",],axis=1)#IDはいらないので削除
    result_df=result_df.replace(0.0000,{"racer_1_ave_st_time":0.22}).copy()#新人のave_st_timeを0.22に
    result_df=result_df.replace(0.0000,{"racer_2_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_3_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_4_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_5_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_6_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_1_doub_win":0.02}).copy()#新人の着に絡む確率ave_st_timeを0.02に(新人の半期の偏差から導出)
    result_df=result_df.replace(0.0000,{"racer_2_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_3_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_4_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_5_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_6_doub_win":0.02}).copy()

    #result_base_df=result_df.copy()
    #result_base_df=get_event_info(result_base_df)#開催の情報について付与する関数(年月日に加えて，何日間の開催かどうかも教えてくれる)

    #ダミー変数化
    result_df_dummie=result_df.copy()
    race_dummie_df=pd.get_dummies(result_df_dummie['number_race'])#number_raceをダミー化
    for column, val in race_dummie_df.iteritems():
        result_df_dummie['race_{}'.format(int(column))]=val
    result_df_dummie=result_df_dummie.drop('number_race',axis=1).copy()

    cols=list(result_df_dummie.columns)
    male_cols=[s for s in cols if 'male' in s]#性別を示すカラムを取り出す

    #===========================新規、性別の取り出し機能が良くなかったため作り直す
    empty_arr=[0]*len(result_df_dummie)
    for col in male_cols:
        for number in np.arange(0,2,1):
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr.copy()
        male_dummie_df=pd.get_dummies(result_df_dummie[col]).copy()#性別をダミー化
        for column, val in male_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val.copy()
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1).copy()

    cols=list(result_df_dummie.columns)



    moter_cols=[s for s in cols if '_mo' in s]#モーター番号を示すカラムを取り出す
    boat_cols=[s for s in cols if '_bo' in s]#ボート番号を示すカラムを取り出す

    #boat、moterの情報は使わない、
    numbers=np.arange(1, 100, 1)
    empty_arr=[0]*len(result_df_dummie)
    for col in moter_cols:
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1).copy()
    for col in boat_cols:
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1).copy()

    #クラスタリング
    #分けてみるクラスタの数は[3,5,7,9]の4個
    #cluster_target_df　　trainのデータからリザルトと配当金を取り除いたもの
    #学習データのdateを年に変換
    result_df_dummie['date']=pd.to_datetime(result_df_dummie['date'])#日付が文字列なのでdateを日付型に変換
    result_df_dummie['year']=result_df_dummie['date'].dt.year
    
    #==========================================================================
    #result_df_dummie=result_df_dummie[result_df_dummie['year']!=2020].copy()#2020のデータを完全に切り離す。
    #==========================================================================
    #クラアスタリング用の学習、予測用のデータの切り分け
    #年，月，日とかの取得
    #now_ym:開始の月
    test_st_date = now_ym - relativedelta(months=range_test_m)#テストデータに使用する区間を決める
    final_test_en_date = now_ym + relativedelta(months=range_final_m)#最終テストの終了日
    
    clustar_final_test_df=result_df_dummie[(result_df_dummie['date']>=now_ym) & (result_df_dummie['date']<final_test_en_date) ].copy()#今の日付以降を最終チェックデータ(予測のターゲット)に。
    clustar_test_df = result_df_dummie[(result_df_dummie['date']<now_ym) & ((result_df_dummie['date']>=test_st_date) )].copy()#今日に日より前の，指定した区間でのテストデータ
    clustar_train_df =  result_df_dummie[(result_df_dummie['date']<test_st_date)].copy()#そのほかを学習データに
    result_df_dummie=result_df_dummie[result_df_dummie['date']<final_test_en_date]
    #年の情報だけ切り分けに使ったからここで消す。
    clustar_final_test_df=clustar_final_test_df.drop('date',axis=1).copy()
    clustar_test_df=clustar_test_df.drop('date',axis=1).copy()
    clustar_train_df=clustar_train_df.drop('date',axis=1).copy()

    #クラスタリングに邪魔だから消したいけど、後々使うものはいったんよけておく
    result=result_df_dummie['result_com'].values.copy()#
    money=result_df_dummie['money'].values.copy()#
    years=result_df_dummie['year'].values.copy()#
    dates=result_df_dummie['date'].values.copy()#
    
    #安全なところに移したら削除する
    result_df_dummie=result_df_dummie.drop('result_com',axis=1)
    result_df_dummie=result_df_dummie.drop('money',axis=1)
    result_df_dummie=result_df_dummie.drop('year',axis=1)
    #esult_df_dummie=result_df_dummie.drop('date',axis=1)
    
    

    target_num_cluster=[3,5,7,9]#分けるクラスタ数によってモデルの名前を変える
    for num_cluster in target_num_cluster:
        Km = KMeans(random_state=7,n_clusters=num_cluster).fit(clustar_train_df)#rondom_stateはラッキーセブン
        #final_test_pred =Km.predict(clustar_final_test_df)#rondom_stateはラッキーセブン
        final_test_pred =Km.predict(clustar_final_test_df)#rondom_stateはラッキーセブン
        test_pred =Km.predict(clustar_test_df)#rondom_stateはラッキーセブン
        train_pred = Km.predict(clustar_train_df)#rondom_stateはラッキーセブン
        #Km=========================実査に使うときはこれのモデルを会場ごとに保存して使用。
        #clustar_final_test_df['num={}'.format(num_cluster)]=final_test_pred
        clustar_final_test_df['num={}'.format(num_cluster)]=final_test_pred
        clustar_test_df['num={}'.format(num_cluster)]=test_pred
        clustar_train_df['num={}'.format(num_cluster)]=train_pred

    #結合して元の形に戻す。
    #clustar_df=pd.concat([clustar_train_df, clustar_test_df,clustar_final_test_df]).copy()
#     clustar_final_test_df['check']='final'#確認用
#     clustar_test_df['check']='test'#確認用
#     clustar_train_df['check']='train'#確認用
    clustar_df=pd.concat([clustar_train_df, clustar_test_df,clustar_final_test_df]).copy()
    clustar_df['year']=years
    clustar_df['date']=dates
    clustar_df['money']=money
    clustar_df['result_com']=result
    model_df=clustar_df.copy()
    return model_df

def model_score_XGboost_th_section_has_final(version,place_name,result_df,now_ym,range_test_m,range_final_m):#XGboostの出力を確率のやつを使用したバージョン、閾値の探索も行う。
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','depth','target_per','threshold','total_get_test', 'total_use_test','num_com_test','num_pred_test','gain_test','gain_std_test','num_hit_test','buy_hit_per_test','buy_hit_per_std_test','plus_month_num_test','diff_mea_med_test','total_get_final', 'total_use_final','num_com_final','num_pred_final','gain_final','gain_std_final','num_hit_final','buy_hit_per_final','buy_hit_per_std_final','plus_month_num_final','diff_mea_med_final'])#スコアを格納するdf

    #学習データの切り分け
    #年，月，日とかの取得
    #now_ym:開始の月
    test_st_date = now_ym - relativedelta(months=range_test_m)#テストデータに使用する区間を決める
    final_test_en_date = now_ym + relativedelta(months=range_final_m)#最終テストの終了日
    #以下学習データ
    final_test_df=result_df[(result_df['date']>=now_ym) & (result_df['date']<final_test_en_date) ].copy()#今の日付以降を最終チェックデータ(予測のターゲット)に。
    test_df = result_df[(result_df['date']<now_ym) & ((result_df['date']>=test_st_date) )].copy()#今日に日より前の，指定した区間でのテストデータ
    train_df =  result_df[(result_df['date']<test_st_date)].copy()#そのほかを学習データに

    #V4ではdateは性能評価用のdf作成時に使用するので別でとっておく(最終的にtransしたpredのdfに結合する)
    final_test_dates=final_test_df['date'].values
    test_dates=test_df['date'].values
    train_dates=train_df['date'].values

    #学習データを切り分けたらyearはいらないから削除する
    final_test_df=final_test_df.drop(['year'],axis=1)
    test_df=test_df.drop(['year'],axis=1)
    train_df=train_df.drop(['year'],axis=1)
    final_test_df=final_test_df.drop(['date'],axis=1)
    test_df=test_df.drop(['date'],axis=1)
    train_df=train_df.drop(['date'],axis=1)

    train_money=pd.Series(train_df['money'])
    test_money=pd.Series(test_df['money'])
    final_test_money=pd.Series(final_test_df['money'])
    test_gain_arr=test_money.values
    final_test_gain_arr=final_test_money.values
    #x,yへの切り分け
    #出現数の分布
    result_com_s=test_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    gain_mean=test_df.groupby('result_com')['money'].mean()
    gain_mean=gain_mean.sort_index()

    gain_median=test_df.groupby('result_com')['money'].median()
    gain_median=gain_median.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index,
                                'result_com_num':result_com_s.values,
                                'result_com_per':result_com_s.values/sum(result_com_s.values)*100,
                                'gain_mean':gain_mean.values,
                                'gain_median':gain_median.values,})
    result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため




    for result_com_number in tqdm(result_com_df['result_com'].values):
        #print(result_com_number)
        result_com=result_com_number
        #result_comごとの閾値の決定========================================================================

        gain_th=10#利益率の閾値
        result_s=result_com_df[result_com_df['result_com']==result_com]
        buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値
        num_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_arr=[0]*len(result_train_df)
        i=0
        for result in result_train_df['result_com']:
            if ((result==result_com)):
                result_arr[i]=1
            else:
                result_arr[i]=0
            i+=1
        result_train_df['result_com']=result_arr

        target_test_df=test_df.copy()
        result_arr=[0]*len(target_test_df)
        i=0
        for result in target_test_df['result_com']:
            if ((result==result_com)):
                result_arr[i]=1
            else:
                result_arr[i]=0
            i+=1
        target_test_df['result_com']=result_arr

        target_final_test_df=final_test_df.copy()
        result_arr=[0]*len(target_final_test_df)
        i=0
        for result in target_final_test_df['result_com']:
            if ((result==result_com)):
                result_arr[i]=1
            else:
                result_arr[i]=0
            i+=1
        target_final_test_df['result_com']=result_arr

        result_train_df['money']=train_money
        target_test_df['money']=test_money
        target_final_test_df['money']=final_test_money
        #学習データラベル変換終わり============================================

        for_arr=np.arange(1,85)
        #for_arr=np.arange(1,100,1)
        accuracy_arr=[0]*len(for_arr)
        target_per_arr=[0]*len(for_arr)
        pred_0=[0]*len(for_arr)
        gain_arr=[0]*len(for_arr)
        model_gain_arr=[0]*len(target_test_df)
        #depths_arr=[4,5,6,7,8]
        #depths_arr=[5,6,8]
        depths_arr=[5,8]
        for depth in depths_arr:
            for sum_target_per in for_arr:

                index=sum_target_per-1
                #target_per=50+sum_target_per
                target_per=100+(sum_target_per*2)
                target_per_arr[index]=target_per

                #モデルの評価指標値を格納するseries======================
                model_score_s=pd.Series(index=['target_com','depth','target_per','threshold','total_get_test', 'total_use_test','num_com_test','num_pred_test','gain_test','gain_std_test','num_hit_test','buy_hit_per_test','buy_hit_per_std_test','plus_month_num_test','diff_mea_med_test','total_get_final', 'total_use_final','num_com_final','num_pred_final','gain_final','gain_std_final','num_hit_final','buy_hit_per_final','buy_hit_per_std_final','plus_month_num_final','diff_mea_med_final'], dtype='float64')
                model_score_s['target_com']=result_com#目標としているresult_comラベル番号
                model_score_s['depth']=depth#ハイパーパラメータ＿木の深さ
                model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
                #======================
                #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
                # 一層目の判別機のtrainデータ　:terget_result_df
                target_df=result_train_df#ベースのデータフレームをコピー
                target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
                target_1_df=target_df[target_df['result_com']==1]
                len_1=len(target_1_df)
                target_0_df=target_df[target_df['result_com']==0]
                len_0=len(target_0_df)
                target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0]#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
                target_train_df=pd.concat([target_1_df, target_0_df])
                #学習＆予測ぱーと========================================================================
                #==========================================================================================================================================
                #データの切り分け
                target_x_train=target_train_df.drop('money',axis=1).copy()
                target_x_train=target_x_train.drop('result_com',axis=1)

                target_x_test=target_test_df.drop('money',axis=1).copy()
                target_x_test=target_x_test.drop('result_com',axis=1)

                target_x_final=target_final_test_df.drop('money',axis=1).copy()
                target_x_final=target_x_final.drop('result_com',axis=1)

                target_y_train=target_train_df['result_com']
                target_y_test=target_test_df['result_com']
                target_y_final=target_final_test_df['result_com']
                train_x, valid_x, train_y, valid_y = train_test_split(target_x_train, target_y_train, test_size=0.2, shuffle=True, random_state=7)#学習データ内でさらに分割してロスをもとに修正をする。

                #XGboostのデータ型に変換する
                train = xgb.DMatrix(train_x, label=train_y)#学習用
                valid = xgb.DMatrix(valid_x, label=valid_y)#学習時のロス修正用
                test = xgb.DMatrix(target_x_test, label=target_y_test)#実際に使った時の利益率の算出用
                final = xgb.DMatrix(target_x_final, label=target_y_final)#最終確認用の区間

                #xgb.config_context(verbosity=0)
                param = {'max_depth': depth, #パラメータの設定
                                 #'eta': 1.8,
                                 #'eta': 0.8,
                                 'eta': 1.3,
                                 #'eta': 0.2,
                                 #'objective': 'binary:hinge',
                                 'objective': 'binary:logistic',#確率で出力
                                 'eval_metric': 'logloss',
                                 'verbosity':0,
                                 'subsample':0.8,
                                 'nthread':10,
                                 'gpu_id':0,
                                 'seed':7,
                                 'tree_method':'gpu_hist'
                                }
                evallist = [(valid, 'eval'), (train, 'train')]#学習時にバリデーションを監視するデータの指定。
                #bst = xgb.train(param, train,num_boost_round=1000,early_stopping_rounds=30)
                #num_round = 10000
                num_round = 400
                bst = xgb.train(param, train,num_round,evallist, early_stopping_rounds=30, verbose_eval=0 )
                #bst = xgb.train(param, train,num_round,evallist, verbose=100,early_stopping_rounds=30 )
                #RF = RandomForestClassifier(random_state=1,n_estimators=1000,max_depth=depth)
                #RF = RF.fit(target_x_train,target_y_train)


                # 未知データに対する予測値
                #predict_y_test = RF.predict(target_x_test)
                predict_y_test=bst.predict(test)
                predict_y_final=bst.predict(final)
                #==========================================================================================================================================
                #==========================================================================================================================================
                #==========================================================================================================================================
                #==========================================================================================================================================

                #[1]の正答率を見る
                pred_test_df=pd.DataFrame({'pred_proba':predict_y_test#確率分布での出力
                                          , 'test':target_y_test})
                pred_final_test_df=pd.DataFrame({'pred_proba':predict_y_final#確率分布での出力
                                          , 'test':target_y_final})


                th_arr=[0.85,0.9,0.92]
                for th in th_arr:
                    trans_test_df=pred_th_trans(pred_test_df,th)
                    trans_final_test_df=pred_th_trans(pred_final_test_df,th)
                    #num_1=len(trans_df[trans_df['test']==1])
                    #追加　配当金の情報も考慮する。
                    count_test=0
                    gain_index=0
                    model_test_gain_arr=[0]*len(test_df)
                    test_hit_arr=[0]*len(test_df)#あたっているかあたっていないかのフラグを格納した配列
                    for _, s in trans_test_df.iterrows():
                        if ((s['pred']==1) and (s['test']==1)):#もし購買しているかつ的中をしていたら・・・
                            count_test+=1#的中回数
                            model_test_gain_arr[gain_index]=test_gain_arr[gain_index]
                            test_hit_arr[gain_index]=1
                        gain_index+=1

                    count_final=0
                    gain_index=0
                    model_final_test_gain_arr=[0]*len(final_test_df)
                    final_test_hit_arr=[0]*len(final_test_df)#あたっているかあたっていないかのフラグを格納した配列
                    for _, s in trans_final_test_df.iterrows():
                        if ((s['pred']==1) and (s['test']==1)):#もし購買しているかつ的中をしていたら・・・
                            count_final+=1#的中回数
                            model_final_test_gain_arr[gain_index]=final_test_gain_arr[gain_index]
                            final_test_hit_arr[gain_index]=1
                        gain_index+=1

    #                     gain_arr[index]=sum(model_gain_arr)
    #                     accuracy_arr[index]=(count/num_1)*100
                    #=================================================
                    try:
                        pred_0[index]=trans_df['pred'].value_counts()[0]
                    except:
                        pred_0[index]=0

                    #V4から増やした集計(月での集計を加えて，comごとでの安定性について確認する)
                    test_analysis_df=trans_test_df.copy()#集計結果を格納するためのdf
                    final_test_analysis_df=trans_final_test_df.copy()#集計結果を格納するためのdf
                    #集計のための情報を結合する
                    test_analysis_df['hit_flag']=test_hit_arr#的中時に１のフラグを結合する
                    test_analysis_df['gain']=model_test_gain_arr#的中して得られたゲインの金額を格納した配列
                    test_analysis_df['date']=test_dates#月ごとの分析のために日付のデータを結合する
                    test_analysis_df=get_event_info(test_analysis_df)#開催情報を取得

                    final_test_analysis_df['hit_flag']=final_test_hit_arr#的中時に１のフラグを結合する
                    final_test_analysis_df['gain']=model_final_test_gain_arr#的中して得られたゲインの金額を格納した配列
                    final_test_analysis_df['date']=final_test_dates#月ごとの分析のために日付のデータを結合する
                    final_test_analysis_df=get_event_info(final_test_analysis_df)#開催情報を取得

                    test_m_analysis=calc_monthly_analysis(test_analysis_df)#月別の分析結果を取得
                    final_test_m_analysis=calc_monthly_analysis(final_test_analysis_df)#月別の分析結果を取得
                    test_m_desc=test_m_analysis.describe()
                    final_test_m_desc=final_test_m_analysis.describe()
                    #scoreのseriesに情報書き込み==================
                    model_score_s['threshold']=th
                    #テストデータ=============================================================
                    #総収益
                    model_score_s['total_get_test']=sum(model_test_gain_arr)
                    #投資金額
                    model_score_s['total_use_test']=100*trans_test_df['pred'].sum()
                    #出現数
                    model_score_s['num_com_test']=sum(target_y_test)
                    #購買予測数
                    model_score_s['num_pred_test']=trans_test_df['pred'].sum()
                    #利益率
                    model_score_s['gain_test']=(model_score_s['total_get_test']/model_score_s['total_use_test'])*100
                    #利益率の標準偏差
                    model_score_s['gain_std_test']=test_m_desc.loc['std','income_per']
                    #的中数
                    model_score_s['num_hit_test']=count_test
                    #購買的中率
                    model_score_s['buy_hit_per_test']=(count_test/trans_test_df['pred'].sum())*100
                    #購買的中率の標準偏差
                    model_score_s['buy_hit_per_std_test']=test_m_desc.loc['std','buy_hit_per']
                    #配当がプラスになった月の数
                    model_score_s['plus_month_num_test']=len(test_m_analysis[test_m_analysis['income']>0])
                    #得られた配当の中央値と平均の差(中央値-平均，つまりマイナスが大きいほど高い配当が引っ張っている)
                    model_score_s['diff_mea_med_test']=(test_m_desc.loc['mean','mean_income'])-(test_m_desc.loc['mean','median_income'])

                    #最終テストデータ(カラム名の説明に関してはテストと同じなので割愛させていただきます)=============================================================
                    model_score_s['total_get_final']=sum(model_final_test_gain_arr)
                    model_score_s['total_use_final']=100*trans_final_test_df['pred'].sum()
                    model_score_s['num_com_final']=sum(target_y_final)
                    model_score_s['num_pred_final']=trans_final_test_df['pred'].sum()
                    model_score_s['gain_final']=(model_score_s['total_get_final']/model_score_s['total_use_final'])*100
                    model_score_s['gain_std_final']=final_test_m_desc.loc['std','income_per']

                    model_score_s['num_hit_final']=count_final
                    model_score_s['buy_hit_per_final']=(count_final/trans_final_test_df['pred'].sum())*100
                    model_score_s['buy_hit_per_std_final']=final_test_m_desc.loc['std','buy_hit_per']
                    model_score_s['plus_month_num_final']=len(final_test_m_analysis[final_test_m_analysis['income']>0])
                    model_score_s['diff_mea_med_final']=(final_test_m_desc.loc['mean','mean_income'])-(final_test_m_desc.loc['mean','median_income'])

                    #dfに書き込み
                    model_score_df=model_score_df.append(model_score_s,ignore_index=True)
    #モデルの「スコアを保存
    dir_path = "../../bot_database/{place_name}/model_score_{place_name}/v4_score/{V}/{place_name}_model_score_st{stDate}_finalM{finalM}_{V}.csv".format(place_name=place_name,V=version,stDate=now_ym.strftime('%Y%m%d'),finalM=range_final_m)#作成したデータの書き込み先#使用するデータの読み込み
    #print(dir_path)
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

def model_score_period_XGboost_th_section_has_final(version,place_name,result_df,now_ym,range_test_m,range_final_m):#昨年同時期のデータを用いてXGboostの出力を確率のやつを使用したバージョン、閾値の探索も行う。
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','depth','target_per','threshold','total_get_test', 'total_use_test','num_com_test','num_pred_test','gain_test','gain_std_test','num_hit_test','buy_hit_per_test','buy_hit_per_std_test','plus_month_num_test','diff_mea_med_test','total_get_final', 'total_use_final','num_com_final','num_pred_final','gain_final','gain_std_final','num_hit_final','buy_hit_per_final','buy_hit_per_std_final','plus_month_num_final','diff_mea_med_final'])#スコアを格納するdf

    #学習データの切り分け
    #年，月，日とかの取得
    #now_ym:開始の月
    test_en_date = now_ym - relativedelta(years=1)+relativedelta(months=range_test_m)#テストデータに使用する区間の終了日を決める(一年前の昨年同時期で三カ月)
    test_st_date = now_ym - relativedelta(years=1)#テストデータに使用する区間の開始を決める(一年前の昨年同時期)
    final_test_en_date = now_ym + relativedelta(months=range_final_m)#最終テストの終了日
    #以下学習データ
    final_test_df=result_df[(result_df['date']>=now_ym) & (result_df['date']<final_test_en_date) ].copy()#今の日付以降を最終チェックデータ(予測のターゲット)に。
    test_df = result_df[(result_df['date']<test_en_date) & ((result_df['date']>=test_st_date) )].copy()#今日より前の，指定した区間でのテストデータ
    train_df =  result_df[(result_df['date']<test_st_date)].copy()#そのほかを学習データに

    #V4ではdateは性能評価用のdf作成時に使用するので別でとっておく(最終的にtransしたpredのdfに結合する)
    final_test_dates=final_test_df['date'].values
    test_dates=test_df['date'].values
    train_dates=train_df['date'].values

    #学習データを切り分けたらyearはいらないから削除する
    final_test_df=final_test_df.drop(['year'],axis=1)
    test_df=test_df.drop(['year'],axis=1)
    train_df=train_df.drop(['year'],axis=1)
    final_test_df=final_test_df.drop(['date'],axis=1)
    test_df=test_df.drop(['date'],axis=1)
    train_df=train_df.drop(['date'],axis=1)

    train_money=pd.Series(train_df['money'])
    test_money=pd.Series(test_df['money'])
    final_test_money=pd.Series(final_test_df['money'])
    test_gain_arr=test_money.values
    final_test_gain_arr=final_test_money.values
    #x,yへの切り分け
    #出現数の分布
    result_com_s=test_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    gain_mean=test_df.groupby('result_com')['money'].mean()
    gain_mean=gain_mean.sort_index()

    gain_median=test_df.groupby('result_com')['money'].median()
    gain_median=gain_median.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index,
                                'result_com_num':result_com_s.values,
                                'result_com_per':result_com_s.values/sum(result_com_s.values)*100,
                                'gain_mean':gain_mean.values,
                                'gain_median':gain_median.values,})
    result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため




    for result_com_number in tqdm(result_com_df['result_com'].values):
        #print(result_com_number)
        result_com=result_com_number
        #result_comごとの閾値の決定========================================================================

        gain_th=10#利益率の閾値
        result_s=result_com_df[result_com_df['result_com']==result_com]
        buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値
        num_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_arr=[0]*len(result_train_df)
        i=0
        for result in result_train_df['result_com']:
            if ((result==result_com)):
                result_arr[i]=1
            else:
                result_arr[i]=0
            i+=1
        result_train_df['result_com']=result_arr

        target_test_df=test_df.copy()
        result_arr=[0]*len(target_test_df)
        i=0
        for result in target_test_df['result_com']:
            if ((result==result_com)):
                result_arr[i]=1
            else:
                result_arr[i]=0
            i+=1
        target_test_df['result_com']=result_arr

        target_final_test_df=final_test_df.copy()
        result_arr=[0]*len(target_final_test_df)
        i=0
        for result in target_final_test_df['result_com']:
            if ((result==result_com)):
                result_arr[i]=1
            else:
                result_arr[i]=0
            i+=1
        target_final_test_df['result_com']=result_arr

        result_train_df['money']=train_money
        target_test_df['money']=test_money
        target_final_test_df['money']=final_test_money
        #学習データラベル変換終わり============================================

        for_arr=np.arange(1,85)
        #for_arr=np.arange(1,100,1)
        accuracy_arr=[0]*len(for_arr)
        target_per_arr=[0]*len(for_arr)
        pred_0=[0]*len(for_arr)
        gain_arr=[0]*len(for_arr)
        model_gain_arr=[0]*len(target_test_df)
        #depths_arr=[4,5,6,7,8]
        #depths_arr=[5,6,8]
        depths_arr=[5,8]
        for depth in depths_arr:
            for sum_target_per in for_arr:

                index=sum_target_per-1
                #target_per=50+sum_target_per
                target_per=100+(sum_target_per*2)
                target_per_arr[index]=target_per

                #モデルの評価指標値を格納するseries======================
                model_score_s=pd.Series(index=['target_com','depth','target_per','threshold','total_get_test', 'total_use_test','num_com_test','num_pred_test','gain_test','gain_std_test','num_hit_test','buy_hit_per_test','buy_hit_per_std_test','plus_month_num_test','diff_mea_med_test','total_get_final', 'total_use_final','num_com_final','num_pred_final','gain_final','gain_std_final','num_hit_final','buy_hit_per_final','buy_hit_per_std_final','plus_month_num_final','diff_mea_med_final'], dtype='float64')
                model_score_s['target_com']=result_com#目標としているresult_comラベル番号
                model_score_s['depth']=depth#ハイパーパラメータ＿木の深さ
                model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
                #======================
                #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
                # 一層目の判別機のtrainデータ　:terget_result_df
                target_df=result_train_df#ベースのデータフレームをコピー
                target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
                target_1_df=target_df[target_df['result_com']==1]
                len_1=len(target_1_df)
                target_0_df=target_df[target_df['result_com']==0]
                len_0=len(target_0_df)
                target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0]#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
                target_train_df=pd.concat([target_1_df, target_0_df])
                #学習＆予測ぱーと========================================================================
                #==========================================================================================================================================
                #データの切り分け
                target_x_train=target_train_df.drop('money',axis=1).copy()
                target_x_train=target_x_train.drop('result_com',axis=1)

                target_x_test=target_test_df.drop('money',axis=1).copy()
                target_x_test=target_x_test.drop('result_com',axis=1)

                target_x_final=target_final_test_df.drop('money',axis=1).copy()
                target_x_final=target_x_final.drop('result_com',axis=1)

                target_y_train=target_train_df['result_com']
                target_y_test=target_test_df['result_com']
                target_y_final=target_final_test_df['result_com']
                train_x, valid_x, train_y, valid_y = train_test_split(target_x_train, target_y_train, test_size=0.2, shuffle=True, random_state=7)#学習データ内でさらに分割してロスをもとに修正をする。

                #XGboostのデータ型に変換する
                train = xgb.DMatrix(train_x, label=train_y)#学習用
                valid = xgb.DMatrix(valid_x, label=valid_y)#学習時のロス修正用
                test = xgb.DMatrix(target_x_test, label=target_y_test)#実際に使った時の利益率の算出用
                final = xgb.DMatrix(target_x_final, label=target_y_final)#最終確認用の区間

                #xgb.config_context(verbosity=0)
                param = {'max_depth': depth, #パラメータの設定
                                 #'eta': 1.8,
                                 #'eta': 0.8,
                                 'eta': 1.3,
                                 #'eta': 0.2,
                                 #'objective': 'binary:hinge',
                                 'objective': 'binary:logistic',#確率で出力
                                 'eval_metric': 'logloss',
                                 'verbosity':0,
                                 'subsample':0.8,
                                 #'nthread':10,
                                 'nthread':5,
                                 'gpu_id':0,
                                 'seed':7,
                                 'tree_method':'gpu_hist'
                                }
                evallist = [(valid, 'eval'), (train, 'train')]#学習時にバリデーションを監視するデータの指定。
                #bst = xgb.train(param, train,num_boost_round=1000,early_stopping_rounds=30)
                #num_round = 10000
                num_round = 400
                bst = xgb.train(param, train,num_round,evallist, early_stopping_rounds=30, verbose_eval=0 )
                #bst = xgb.train(param, train,num_round,evallist, verbose=100,early_stopping_rounds=30 )
                #RF = RandomForestClassifier(random_state=1,n_estimators=1000,max_depth=depth)
                #RF = RF.fit(target_x_train,target_y_train)


                # 未知データに対する予測値
                #predict_y_test = RF.predict(target_x_test)
                predict_y_test=bst.predict(test)
                predict_y_final=bst.predict(final)
                #==========================================================================================================================================
                #==========================================================================================================================================
                #==========================================================================================================================================
                #==========================================================================================================================================

                #[1]の正答率を見る
                pred_test_df=pd.DataFrame({'pred_proba':predict_y_test#確率分布での出力
                                          , 'test':target_y_test})
                pred_final_test_df=pd.DataFrame({'pred_proba':predict_y_final#確率分布での出力
                                          , 'test':target_y_final})


                th_arr=[0.85,0.9,0.92]
                for th in th_arr:
                    trans_test_df=pred_th_trans(pred_test_df,th)
                    trans_final_test_df=pred_th_trans(pred_final_test_df,th)
                    #num_1=len(trans_df[trans_df['test']==1])
                    #追加　配当金の情報も考慮する。
                    count_test=0
                    gain_index=0
                    model_test_gain_arr=[0]*len(test_df)
                    test_hit_arr=[0]*len(test_df)#あたっているかあたっていないかのフラグを格納した配列
                    for _, s in trans_test_df.iterrows():
                        if ((s['pred']==1) and (s['test']==1)):#もし購買しているかつ的中をしていたら・・・
                            count_test+=1#的中回数
                            model_test_gain_arr[gain_index]=test_gain_arr[gain_index]
                            test_hit_arr[gain_index]=1
                        gain_index+=1

                    count_final=0
                    gain_index=0
                    model_final_test_gain_arr=[0]*len(final_test_df)
                    final_test_hit_arr=[0]*len(final_test_df)#あたっているかあたっていないかのフラグを格納した配列
                    for _, s in trans_final_test_df.iterrows():
                        if ((s['pred']==1) and (s['test']==1)):#もし購買しているかつ的中をしていたら・・・
                            count_final+=1#的中回数
                            model_final_test_gain_arr[gain_index]=final_test_gain_arr[gain_index]
                            final_test_hit_arr[gain_index]=1
                        gain_index+=1

    #                     gain_arr[index]=sum(model_gain_arr)
    #                     accuracy_arr[index]=(count/num_1)*100
                    #=================================================
                    try:
                        pred_0[index]=trans_df['pred'].value_counts()[0]
                    except:
                        pred_0[index]=0

                    #V4から増やした集計(月での集計を加えて，comごとでの安定性について確認する)
                    test_analysis_df=trans_test_df.copy()#集計結果を格納するためのdf
                    final_test_analysis_df=trans_final_test_df.copy()#集計結果を格納するためのdf
                    #集計のための情報を結合する
                    test_analysis_df['hit_flag']=test_hit_arr#的中時に１のフラグを結合する
                    test_analysis_df['gain']=model_test_gain_arr#的中して得られたゲインの金額を格納した配列
                    test_analysis_df['date']=test_dates#月ごとの分析のために日付のデータを結合する
                    test_analysis_df=get_event_info(test_analysis_df)#開催情報を取得

                    final_test_analysis_df['hit_flag']=final_test_hit_arr#的中時に１のフラグを結合する
                    final_test_analysis_df['gain']=model_final_test_gain_arr#的中して得られたゲインの金額を格納した配列
                    final_test_analysis_df['date']=final_test_dates#月ごとの分析のために日付のデータを結合する
                    final_test_analysis_df=get_event_info(final_test_analysis_df)#開催情報を取得

                    test_m_analysis=calc_monthly_analysis(test_analysis_df)#月別の分析結果を取得
                    final_test_m_analysis=calc_monthly_analysis(final_test_analysis_df)#月別の分析結果を取得
                    test_m_desc=test_m_analysis.describe()
                    final_test_m_desc=final_test_m_analysis.describe()
                    #scoreのseriesに情報書き込み==================
                    model_score_s['threshold']=th
                    #テストデータ=============================================================
                    #総収益
                    model_score_s['total_get_test']=sum(model_test_gain_arr)
                    #投資金額
                    model_score_s['total_use_test']=100*trans_test_df['pred'].sum()
                    #出現数
                    model_score_s['num_com_test']=sum(target_y_test)
                    #購買予測数
                    model_score_s['num_pred_test']=trans_test_df['pred'].sum()
                    #利益率
                    model_score_s['gain_test']=(model_score_s['total_get_test']/model_score_s['total_use_test'])*100
                    #利益率の標準偏差
                    model_score_s['gain_std_test']=test_m_desc.loc['std','income_per']
                    #的中数
                    model_score_s['num_hit_test']=count_test
                    #購買的中率
                    model_score_s['buy_hit_per_test']=(count_test/trans_test_df['pred'].sum())*100
                    #購買的中率の標準偏差
                    model_score_s['buy_hit_per_std_test']=test_m_desc.loc['std','buy_hit_per']
                    #配当がプラスになった月の数
                    model_score_s['plus_month_num_test']=len(test_m_analysis[test_m_analysis['income']>0])
                    #得られた配当の中央値と平均の差(中央値-平均，つまりマイナスが大きいほど高い配当が引っ張っている)
                    model_score_s['diff_mea_med_test']=(test_m_desc.loc['mean','mean_income'])-(test_m_desc.loc['mean','median_income'])

                    #最終テストデータ(カラム名の説明に関してはテストと同じなので割愛させていただきます)=============================================================
                    model_score_s['total_get_final']=sum(model_final_test_gain_arr)
                    model_score_s['total_use_final']=100*trans_final_test_df['pred'].sum()
                    model_score_s['num_com_final']=sum(target_y_final)
                    model_score_s['num_pred_final']=trans_final_test_df['pred'].sum()
                    model_score_s['gain_final']=(model_score_s['total_get_final']/model_score_s['total_use_final'])*100
                    model_score_s['gain_std_final']=final_test_m_desc.loc['std','income_per']

                    model_score_s['num_hit_final']=count_final
                    model_score_s['buy_hit_per_final']=(count_final/trans_final_test_df['pred'].sum())*100
                    model_score_s['buy_hit_per_std_final']=final_test_m_desc.loc['std','buy_hit_per']
                    model_score_s['plus_month_num_final']=len(final_test_m_analysis[final_test_m_analysis['income']>0])
                    model_score_s['diff_mea_med_final']=(final_test_m_desc.loc['mean','mean_income'])-(final_test_m_desc.loc['mean','median_income'])

                    #dfに書き込み
                    model_score_df=model_score_df.append(model_score_s,ignore_index=True)
    #モデルの「スコアを保存
    dir_path = "../../bot_database/{place_name}/model_score_{place_name}/v4_score/{V}/{place_name}_model_score_period_st{stDate}_finalM{finalM}_{V}.csv".format(place_name=place_name,V=version,stDate=now_ym.strftime('%Y%m%d'),finalM=range_final_m)#作成したデータの書き込み先#使用するデータの読み込み
    #print(dir_path)
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None


### 機能チェック(いったん芦屋だけで動かしてみる)

In [10]:
#設定する変数=================================================================


place_master=master.get_place_master()
place_name='asiya'
#設定する変数=================================================================


result_filepath="../..//bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
result_base_df=pd.read_csv(result_filepath)
result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)
for i in range(4):
    now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1)
    print('st_date:',now_ym,'=======================================')
    #now_ym=datetime.datetime(year=2020, month=1,day=1)#スタート地点(今)の年月を指定，これを基準にさかのぼっていってテスト,学習データを決める(配列の中身は[year,month])
    range_test_m=5#テストデータに使用する月の数を指定
    range_final_m=5#最終的な評価に使うデータの数(今はここで５となっているが実際に評価をする際に必要な数は大きい値位にしておけば後で調整ができる)

    result_df=data_making_clustar_section_has_final(result_base_df,now_ym,range_test_m,range_final_m)#モデル関連に使用するdfの作成関数(クラスタリングあり、モータ番号、艇番号なし)（加工関数）
    model_score_XGboost_th_section_has_final(version,place_name,result_df,now_ym,range_test_m,range_final_m)
    
print('DONE')
    #display(result_df)


st_date: 2020-01-01 00:00:00 =======================================


  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

asiya


 89%|██████████████████████████████████████████████████████████████████████▌        | 25/28 [2:05:01<13:29, 270.00s/it]<ipython-input-8-f5d39244789d>:90: RuntimeWarning: invalid value encountered in true_divide
  income_per=(get_m/use_m)*100
100%|███████████████████████████████████████████████████████████████████████████████| 28/28 [2:16:22<00:00, 292.21s/it]


st_date: 2020-04-01 00:00:00 =======================================


...

st_date: 2020-10-01 00:00:00 =======================================


  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

asiya


100%|███████████████████████████████████████████████████████████████████████████████| 28/28 [2:02:47<00:00, 263.14s/it]

DONE


## 区間変更バージョンも作る(場所は同じく芦屋，なおテスト区間は５カ月，ファイナル区間は３カ月)
＝１０月スタートだけデータが少なくなってしまって同じロジックが適用できない，，

In [11]:
#設定する変数=================================================================
version='V4_1'#バージョン

place_master=master.get_place_master()
place_name='asiya'
#設定する変数=================================================================


result_filepath="../..//bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
result_base_df=pd.read_csv(result_filepath)
result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)
for i in range(4):
    now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1)
    print('st_date:',now_ym,'=======================================')
    #now_ym=datetime.datetime(year=2020, month=1,day=1)#スタート地点(今)の年月を指定，これを基準にさかのぼっていってテスト,学習データを決める(配列の中身は[year,month])
    range_test_m=5#テストデータに使用する月の数を指定
    range_final_m=3#最終的な評価に使うデータの数(今はここで５となっているが実際に評価をする際に必要な数は大きい値位にしておけば後で調整ができる)

    result_df=data_making_clustar_section_has_final(result_base_df,now_ym,range_test_m,range_final_m)#モデル関連に使用するdfの作成関数(クラスタリングあり、モータ番号、艇番号なし)（加工関数）
    model_score_XGboost_th_section_has_final(version,place_name,result_df,now_ym,range_test_m,range_final_m)
    
print('DONE')
    #display(result_df)


st_date: 2020-01-01 00:00:00 =======================================


  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

asiya


 89%|██████████████████████████████████████████████████████████████████████▌        | 25/28 [1:55:22<12:56, 258.92s/it]<ipython-input-4-f5d39244789d>:90: RuntimeWarning: invalid value encountered in true_divide
  income_per=(get_m/use_m)*100
100%|███████████████████████████████████████████████████████████████████████████████| 28/28 [2:06:02<00:00, 270.08s/it]


st_date: 2020-04-01 00:00:00 =======================================


...

st_date: 2020-10-01 00:00:00 =======================================


  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

asiya


100%|███████████████████████████████████████████████████████████████████████████████| 28/28 [2:12:15<00:00, 283.41s/it]


DONE


# 昨年同時期バージョン(period)
（ex:20200101がnowdateの場合，，）<br>
test:20190101---20190401<br>
final:20200101--20200401<br>
train:  ～20281201

In [14]:
#設定する変数=================================================================
version='V4_1'#バージョン

place_master=master.get_place_master()
place_name='asiya'
#設定する変数=================================================================


result_filepath="../..//bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
result_base_df=pd.read_csv(result_filepath)
result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)
for i in range(4):
    now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1)
    print('st_date:',now_ym,'=======================================')
    #now_ym=datetime.datetime(year=2020, month=1,day=1)#スタート地点(今)の年月を指定，これを基準にさかのぼっていってテスト,学習データを決める(配列の中身は[year,month])
    range_test_m=3#テストデータに使用する月の数を指定
    range_final_m=3#最終的な評価に使うデータの数(今はここで５となっているが実際に評価をする際に必要な数は大きい値位にしておけば後で調整ができる)

    result_df=data_making_clustar_section_has_final(result_base_df,now_ym,range_test_m,range_final_m)#モデル関連に使用するdfの作成関数(クラスタリングあり、モータ番号、艇番号なし)（加工関数）
    model_score_period_XGboost_th_section_has_final(version,place_name,result_df,now_ym,range_test_m,range_final_m)
    
print('DONE')
    #display(result_df)


st_date: 2020-01-01 00:00:00 =======================================


  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

asiya


100%|███████████████████████████████████████████████████████████████████████████████| 28/28 [1:54:39<00:00, 245.69s/it]


st_date: 2020-04-01 00:00:00 =======================================


...

st_date: 2020-10-01 00:00:00 =======================================


  0%|                                                                                           | 0/28 [00:00<?, ?it/s]

asiya


 93%|█████████████████████████████████████████████████████████████████████████▎     | 26/28 [1:48:23<07:32, 226.08s/it]<ipython-input-10-f5d39244789d>:90: RuntimeWarning: invalid value encountered in true_divide
  income_per=(get_m/use_m)*100
100%|███████████████████████████████████████████████████████████████████████████████| 28/28 [1:55:03<00:00, 246.54s/it]

DONE


In [121]:
version='V4_1'#バージョン
now_ym=datetime.datetime(year=2020, month=1,day=1)#スタート地点(今)の年月を指定，これを基準にさかのぼっていってテスト,学習データを決める(配列の中身は[year,month])
range_test_m=5#テストデータに使用する月の数を指定
range_final_m=5#最終的な評価に使うデータの数(今はここで５となっているが実際に評価をする際に必要な数は大きい値位にしておけば後で調整ができる)
place_master=master.get_place_master()
place_name='asiya'

#def model_score_XGboost_th_section_has_final(version,place_name,result_df,now_ym,range_test_m,range_final_m):#XGboostの出力を確率のやつを使用したバージョン、閾値の探索も行う。
dir_path = "../../bot_database/{place_name}/model_score_{place_name}/v4_score/{place_name}_model_score_st{stDate}_{V}.csv".format(place_name=place_name,V=version,stDate=now_ym.strftime('%Y%m%d'))#作成したデータの書き込み先#使用するデータの読み込み
dir_path   


'../../bot_database/asiya/model_score_asiya/v4_score/asiya_model_score_st20200101_V4_1.csv'

In [114]:
model_score_df.to_csv('csv/asiya_ex.csv', encoding='utf_8_sig')

In [110]:
model_score_df

,target_com,depth,target_per,threshold,total_get_test,total_use_test,num_com_test,num_pred_test,gain_test,gain_std_test,...,num_com_final,num_pred_final,gain_final,gain_std_final,num_hit_final,buy_hit_per_final,buy_hit_per_std_final,diff_mea_med_final,plus_month_num_final,plus_month_num_test
0,1.0,5.0,102.0,0.85,24140.0,29700.0,22543.0,297.0,81.279461,49.191472,...,22364.0,297.0,100.437710,31.332901,37.0,12.457912,0.970024,110.960101,4.0,2.0
1,1.0,5.0,102.0,0.90,22640.0,27600.0,22543.0,276.0,82.028986,47.842818,...,22364.0,277.0,102.310469,41.024556,35.0,12.635379,1.052518,127.272222,4.0,2.0
2,1.0,5.0,102.0,0.92,22640.0,26600.0,22543.0,266.0,85.112782,51.492222,...,22364.0,266.0,93.120301,45.356422,33.0,12.406015,1.083072,70.083333,3.0,2.0
3,1.0,5.0,104.0,0.85,21240.0,27600.0,22543.0,276.0,76.956522,50.287162,...,22364.0,270.0,115.296296,42.007690,34.0,12.592593,0.884247,287.916667,3.0,3.0
4,1.0,5.0,104.0,0.90,19220.0,26400.0,22543.0,264.0,72.803030,48.121798,...,22364.0,251.0,107.968127,58.600878,31.0,12.350598,1.031249,176.285714,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.0,5.0,206.0,0.90,15150.0,14300.0,22543.0,143.0,105.944056,62.067236,...,22364.0,154.0,139.155844,70.768224,23.0,14.935065,0.984789,14.552381,4.0,2.0
158,1.0,5.0,206.0,0.92,14810.0,13100.0,22543.0,131.0,113.053435,63.646672,...,22364.0,141.0,131.063830,71.077784,19.0,13.475177,0.924710,8.333333,3.0,2.0
159,1.0,5.0,208.0,0.85,7690.0,18700.0,22543.0,187.0,41.122995,26.704037,...,22364.0,150.0,87.266667,77.401831,18.0,12.000000,1.534782,6.000000,2.0,0.0
160,1.0,5.0,208.0,0.90,6900.0,16600.0,22543.0,166.0,41.566265,31.173842,...,22364.0,137.0,86.861314,70.251188,17.0,12.408759,1.513609,12.533333,2.0,0.0


In [115]:
pred_test_df

,pred_proba,test
16748,0.000480,0
16749,0.471421,0
16750,0.000002,0
16751,0.013508,1
16752,0.002222,1
...,...,...
16201,0.101803,0
16202,0.005151,0
16203,0.999900,0
16204,0.877730,0


In [120]:
now_ym
test_st_date = now_ym - relativedelta(months=range_test_month) 
test_st_date
tstr = test_st_date.strftime('%Y%m%d')
tstr

'20190801'

## モデルのスコアシートの作成